In [9]:
# %load questions/get_problem.py
import logging
from questions.models import Problem, Solution, TestCase
import json

LOGGER = logging.getLogger(__name__)

def get_problem(problem_id):
    #try:
    # Get problem, test cases and solution
    problem = Problem.objects.get(pk=problem_id)
    LOGGER.debug("Got problem %d", problem.id)
    test_case = TestCase.objects.filter(problem=problem)
    test_case = [json.loads(test.content) for test in test_case]
    LOGGER.debug("Got test cases %s for problem %d", test_case, problem.id)
    solution = Solution.objects.filter(problem=problem, ignore=False).order_by('?')[0]

    # Transform solution into python function
    function_obj = compile(solution.content, 'solution', 'exec')
    exec(function_obj)

    # For each test case, get expected output
    expected_results = []
    for args in test_case:
        expected_results.append(str(eval(solution.header)(*args)))
    #print(expected_results)
    #except Problem.DoesNotExist:
        #raise Problem.DoesNotExist

    context = {
        "problem": problem,
        "test_case": test_case,
        "expected_results": expected_results,
        "tip": solution.tip,
        "header": solution.header
    }
    return context


In [10]:
context = get_problem(730)
# test_case, solution = get_problem(728)

In [20]:
solution.content

'def calculate_determinante(a,b,c):\r\n    det = b**2 - 4*a*c\r\n    return det\r\n\r\ndef calcular_raiz(a,b,c):\r\n    det = calculate_determinante(a,b,c)\r\n    if det > 0:\r\n        raiz1 = (-b + det**(1.0/2))/(2*a)\r\n        raiz2 = (-b - det**(1.0/2))/(2*a)\r\n        return 2\r\n    elif det == 0:\r\n        raiz1 = (-b + det**(1/2))/(2*a)\r\n        return 1\r\n    else:\r\n        return 0'

In [21]:
function_obj = compile(solution.content, 'solution', 'exec')
exec(function_obj)

In [23]:
for args in test_case:
    print(args)
    eval(solution.header)(*args)

[8, 8, 5]
[4, 8, 4]
[2, 3, 1]


In [11]:
user_passed = UserLog.objects.filter(user__username='lmoraes@cos.ufrj.br', outcome__in=["P", "S"]).values_list('problem', flat=True)
problem_id = Problem.objects.filter(chapter__isnull=False).exclude(id__in=user_passed).order_by('id')[0].pk
problem_id


724